# Deep Learning - Project

## Parking Space Detection

### Group 02

# 0. Imports & Global Constants

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET

ORIGINAL_DATASET_PATH = "./datasets/original"
TRANSFORMED_DATASET_PATH = "./datasets/transformed"
TRAINING_DATASET_PATH = "./datasets/training"
VALIDATION_DATASET_PATH = "./datasets/validation"
TESTING_DATASET_PATH = "./datasets/testing"

TRAINING_RATIO = 0.8
VALIDATION_RATIO = 0.1
TESTING_RATIO = 0.1

GITKEEP = '.gitkeep'

IMAGE_WIDTH = 1280
IMAGE_HEIGHT = 720

CLASS_MAPPING = {"0": 0, "1": 1}

# 1. Data Pipeline

# 1.1. Transforming

First we start by transforming our original dataset

In [ ]:
for root, _, files in os.walk(ORIGINAL_DATASET_PATH):
    for file in files:
        source_file_path = os.path.join(root, file)
        destination_file_path = os.path.join(TRANSFORMED_DATASET_PATH, file)
        shutil.move(source_file_path, destination_file_path)

## 1.2. Create the Labels

We'll convert the labels from XML to TXT

In [ ]:
xml_files = [f for f in os.listdir(TRANSFORMED_DATASET_PATH) if f.endswith(".xml")]

print(xml_files)

for xml_file in xml_files:
    xml_path = os.path.join(TRANSFORMED_DATASET_PATH, xml_file)
    print(xml_path)

    tree = ET.parse(xml_path)
    root = tree.getroot()

    txt_filename = os.path.splitext(xml_file)[0] + ".txt"
    txt_path = os.path.join(TRANSFORMED_DATASET_PATH, txt_filename)
    print(txt_filename)
    print(txt_path)
    with open(txt_path, "w") as txt_file:
        for space in root.findall("space"):
            occupied = space.get("occupied")
            class_index = CLASS_MAPPING.get(occupied, -1)
            
            if class_index == -1:
                continue
            
            rotated_rect = space.find("rotatedRect")
            center = rotated_rect.find("center")
            size = rotated_rect.find("size")
            
            center_x = float(center.get("x"))
            center_y = float(center.get("y"))
            width = float(size.get("w"))
            height = float(size.get("h"))
            
            x_center = center_x / IMAGE_WIDTH
            y_center = center_y / IMAGE_HEIGHT
            w = width / IMAGE_WIDTH
            h = height / IMAGE_HEIGHT
            
            txt_file.write(f"{class_index} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
        new_xml_path = os.path.join(TRANSFORMED_DATASET_PATH, xml_file)
        print(new_xml_path)
    os.remove(xml_path)


# Train Test Split

In [ ]:
image_files = [f for f in os.listdir(TRANSFORMED_DATASET_PATH) if f.endswith(".jpg")]
print(image_files)

# num_samples = len(image_files)
# num_train = int(train_ratio * num_samples)
# num_test = int(test_ratio * num_samples)
# num_val = num_samples - num_train - num_test